### sử dụng với dữ liệu creditcard https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download

In [66]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score  
from sklearn.metrics import accuracy_score  
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [67]:
df = pd.read_csv('creditcard.csv')

def create_datastream_by_time(data, time_interval):
    datastream = []
    start_time = int(data['Time'].min())
    end_time = int(data['Time'].max())
    
    for t in range(start_time, end_time + 1, time_interval):
        block = data[(data['Time'] >= t) & (data['Time'] < t + time_interval)]
        if not block.empty:
            datastream.append(block)
    
    return datastream
time_interval = 5000
S = create_datastream_by_time(df, time_interval)

In [68]:
def train_random_forest_classifier(X, y):
    """
    Huấn luyện bộ phân loại Random Forest trên dữ liệu X, y.
    """
    class_count = np.bincount(y)

    # Kiểm tra số lượng lớp có trong y
    if len(class_count) < 2 :
        rf = RandomForestClassifier(n_estimators=10, class_weight={0:1})
        rf.fit(X, y)
        return rf 
    else:
        class_ratio = class_count[1] / class_count[0]
        lambda_min = 1 
        lambda_value = lambda_min + (1 + class_ratio) * lambda_min

        # Huấn luyện bộ phân loại Random Forest
        rf = RandomForestClassifier(n_estimators=10, class_weight={0: 1, 1: lambda_value})
        rf.fit(X, y)
        return rf  # Trả về mô hình đã huấn luyện


In [69]:
def adaptive_ensemble_size(C, sample, unique_label, anpha, min_num = 3):
    """
    Cơ chế adaptive_esemble 
    C : Bộ phân loại yếu
    sample
    unique_label : Tập hợp nhãn trong luồng dữ liệu
    anpha : hyper parameter
    min_num : số lượng C tối thiểu 
    """
    probability_list = []
    for i in range(len(unique_label)):
        li = [] 
        for j in range(len(C)):
            probabilities = C[0].predict_proba(sample)[0]
            probability_dict = {label: prob for label, prob in zip(C[0].classes_, probabilities)}
            latest_proba = probability_dict.get(unique_label[i], 0)
            probabilities1 = C[j].predict_proba(sample)[0]
            probability_dict1 = {label: prob for label, prob in zip(C[j].classes_, probabilities1)}
            current_proba = probability_dict1.get(unique_label[i], 0)
            if len(li) < min_num:
                li.append(current_proba)
            else:
                if abs(current_proba - latest_proba) < anpha :
                    li.append(current_proba)
                else: 
                    break   
        probability_list.append(li)     
    return probability_list

In [70]:
def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def tendency_prediction(probability_list, Y, epsilon=0.01):
    """
    Thực hiện cơ chế dự đoán xu hướng
    Input:
    - probability_list: Danh sách xác suất cho mỗi mẫu
    - Y: Danh sách các lớp trong luồng dữ liệu
    - epsilon: Hệ số trọng số (mặc định là 0.01)
    Output:
    - Ps: Lớp dự đoán cho mẫu
    """
    predicted_probabilities = []
    for i in range(len(Y)):
        li = probability_list[i]
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        slope, intercept = linear_regression(x, y)
        next_value = slope * (len(li) + 1) + intercept
        li.append(next_value)
        weighted_prob = sum([li[x] * (1 + (x + 1) * epsilon) for x in range(len(li))]) / len(li)
        predicted_probabilities.append(weighted_prob)
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps


In [71]:
m = 15
k = 3
unique_labels = set(df.iloc[:, -1])
unique_labels = list(unique_labels)

b1 = pd.concat(S[:m], ignore_index=True)
minority_samples = b1[b1['Class'] == 1].drop('Class', axis=1)
    # Tạo K-means
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(minority_samples)
centroids = kmeans.cluster_centers_

    # Tạo thêm các mẫu gần cụm
new_samples = []
for centroid in centroids:
        for _ in range(4 // 2):
            noise = np.random.normal(0, 0.1, centroid.shape)
            new_sample = centroid + noise
            new_samples.append(new_sample)

new_samples_df = pd.DataFrame(new_samples, columns=minority_samples.columns)
new_samples_df['Class'] = 1

b1_augmented = pd.concat([b1, new_samples_df], ignore_index=True)

class_1_rows = b1_augmented[b1_augmented['Class'] == 1]
num_class_1_rows = class_1_rows.shape[0]

    # Smote
X = b1_augmented.drop('Class', axis=1)
y = b1_augmented['Class']



smote = SMOTE(sampling_strategy={1:num_class_1_rows}, random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

b1_augmented = pd.DataFrame(X_smote, columns=X.columns)
b1_augmented['Class'] = y_smote

unique_labels = list(set(y_smote))

    # Tạo danh sách tất cả các block, bao gồm b1
S = [b1_augmented] + S[m:]

def process_data_stream(S, m, k, unique_labels):
    """
    Processes data stream S using a weak Random Forest classifier.
    - S: Data stream (list of data blocks Bi)
    - m: Max size of classifier set C
    - k: Min size of C to make predictions
    """
    C = deque(maxlen=m) 
    pre = []
    class_1_samples = pd.DataFrame()
    true_labels = []
    for i in range(10):
        Bi = S[i]
        class_1_new_samples = Bi[Bi['Class'] == 1]
        class_1_samples = pd.concat([class_1_samples, class_1_new_samples], ignore_index=True)

        if len(class_1_samples) > m:
            class_1_samples = class_1_samples.iloc[-m:]  # Giữ lại m mẫu mới nhất
        if i + 1 < len(S):
            next_block = S[i + 1]
            class_1_samples_df = class_1_samples
            S[i + 1] = pd.concat([next_block, class_1_samples_df], ignore_index=True)
        block_predictions = []
        X = Bi.iloc[:, :-1] 
        y = Bi.iloc[:, -1]   
        Ci = train_random_forest_classifier(X, y)
        C.append(Ci)
        if len(C) < k:
            continue
        Bi_1 = S[i + 1]
        y = Bi_1.iloc[:, -1]
        for index, row in Bi_1.iterrows():
            sample = pd.DataFrame([row[:-1]], columns=Bi.columns[:-1])
            anpha = (1500 / len(Bi)) * 0.2
            selected_classifiers = adaptive_ensemble_size(C, sample, unique_labels, anpha)
            pre_sample = tendency_prediction(selected_classifiers, unique_labels)
            block_predictions.append(pre_sample)
        pre.append(block_predictions)
        true_labels.append(list(y))
    return C, pre, true_labels
C, pre, true_labels = process_data_stream(S, m, k, unique_labels)
all_predictions = [pred for block in pre for pred in block]  # Tất cả dự đoán
all_true_labels = [label for block in true_labels for label in block]  # Tất cả nhãn đúng

# Tính toán chỉ số tổng
if len(all_predictions) > 0 and len(all_true_labels) > 0:
    precision = precision_score(all_true_labels, all_predictions, average='binary', pos_label=1)
    recall = recall_score(all_true_labels, all_predictions, average='binary', pos_label=1)
    f1 = f1_score(all_true_labels, all_predictions, average='binary', pos_label=1)
    accuracy = accuracy_score(all_true_labels, all_predictions)
    
    print("Total Metrics:")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")

    print(f"  F1 Score: {f1:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
else:
    print("No predictions or true labels available to compute metrics.")

c:\Users\MyPC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 